# Data handling

In this notebook, we will work with the following:

- Reading data with `pandas`.
- Cleaning and transforming data.
- Viewing and selecting data.
- Merging and querying.
- Exporting.

In [1]:
import pandas as pd

# Reading data

`pandas` supports a number of formats that we often find ourselves using.
For example, I often use data in the Stata `dta` and SAS `sas7bdat` formats.
In particular, if you find yourself putting full datasets from WRDS (especially the ones that are not accessible with web forms), you will end up using the SAS format.

`pandas` also handles formats like Excel `xlsx`, comma separated values `csv` (and, indeed, nearly any delimited file), and fixed width data.
The acquisition database, SDC Platinum, has a somewhat unreliable Excel output feature, and the `pandas` fixed width format reader takes nearly all of the pain out of reading in data exported that way.

Note: `pandas` can also write many of the formats that it can read.
A notable exception is `sas7bdat` because it is proprietary and undocumented.
The reader was written with some clever reverse engineering, but writing a valid file is difficult and probably not coming in the future (see [Github issue](https://github.com/pandas-dev/pandas/issues/13031)).
An easy workaround is using the SAS open format `xpt` or `csv`.

In [2]:
# Stata data
firmyear = pd.read_stata('../data/firmyear.dta')
firmyear.head()

,count_of_employees,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017


# Cleaning data

You are likely familiar with a number of data cleaning issues.
However, you may not yet know how to map on what you know in another program to Python.
The pandas documentation has a number of comparison references, including [R](https://pandas.pydata.org/pandas-docs/stable/comparison_with_r.html), [Stata](https://pandas.pydata.org/pandas-docs/stable/comparison_with_stata.html) and [SAS](https://pandas.pydata.org/pandas-docs/stable/comparison_with_sas.html).

Some brief examples are below.

## Data types

In [3]:
firmyear.dtypes

count_of_employees    object
name                  object
year                  object
dtype: object

Note that all of the columns above are of type `object`, which often means that they are strings.
We want to change the things that we know are numbers (i.e. `count_of_employees` and `year`) into the appropriate types (both `int` in this case).

In [4]:
firmyear['year'] = firmyear['year'].astype('int')
firmyear['count_of_employees'] = firmyear['count_of_employees'].astype('int')

# Note, a more general version would be:
# cols = firmyear.columns.drop('name')
# firmyear[cols] = firmyear[cols].apply(pd.to_numeric, errors='coerce')

In [5]:
firmyear

,count_of_employees,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017
5,76,Google,2018


In [6]:
firmyear.dtypes

count_of_employees     int64
name                  object
year                   int64
dtype: object

## Renaming columns

We could rename columns by creating a new column with the correct name and dropping the prior one, but this is more efficient and easily extended to the multiple column case.

In [7]:
# An example of using dictionaries.
_COLUMNS = {
    'count_of_employees': 'size_emp'
}

firmyear = firmyear.rename(columns=_COLUMNS)

In [8]:
firmyear

,size_emp,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017
5,76,Google,2018


## Transformations

We can also do transformations that apply some sort of function or method to data by groups.
This is a fairly simple example, but `pandas` makes it fairly easy to do sophisticated transformations.
See the [split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) documentation.
This is a big topic, and, like before, we are only scratching the surface.

In [9]:
# We can do per-group things like calculating differences.
firmyear['size_emp_change'] = firmyear.groupby(firmyear['name'])['size_emp'].diff()

In [10]:
firmyear

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0
5,76,Google,2018,2.0


# Viewing and selecting data

pandas has a number of tools for viewing and selecting data.
The one we see above is the `df.head()` method that displays the first five rows at the top (or head) of the data.

In [11]:
firmyear.head()

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0


In [12]:
# We can give it a parameter to modify the number of rows.
# Here, we only have six rows, so that's all we get.
firmyear.head(8)

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0
5,76,Google,2018,2.0


In [13]:
# The len() function works on dataframes.
len(firmyear)

6

We can also select one or more columns by using indexing that is somewhat like what we did with dictionaries earlier.
However, we can give the indexer a list, and get the named columns.

Note that, when we ask for one column, pandas gives us a series, not a dataframe, so the display is a little less fancy.

In [14]:
firmyear['name']

0    Microsoft
1    Microsoft
2    Microsoft
3       Google
4       Google
5       Google
Name: name, dtype: object

In [15]:
# Note the two sets of brackets.
# The outer set is for the indexing syntax.
# The inner set is for the list that we're asking the indexer for.
firmyear[['name', 'year']]

,name,year
0,Microsoft,2016
1,Microsoft,2017
2,Microsoft,2018
3,Google,2016
4,Google,2017
5,Google,2018


We can also ask for rows that meet certain conditions.

In [16]:
firmyear[firmyear['name'] == 'Microsoft']

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0


In [17]:
# Note that the expression used for indexing is returning a series of boolean values.
firmyear['name'] == 'Microsoft'

0     True
1     True
2     True
3    False
4    False
5    False
Name: name, dtype: bool

In [18]:
# We can use compound statements that return one boolean value per row.
# Here, it's name == Microsoft or the year is less than 2018.
firmyear[(firmyear['name'] == 'Microsoft') | (firmyear['year'] < 2018)]

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0


In [19]:
# Series have methods for checking whether they're NA.
# This is true if each row is not NA.
firmyear[firmyear['size_emp_change'].notna()]

,size_emp,name,year,size_emp_change
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
4,74,Google,2017,2.0
5,76,Google,2018,2.0


In [20]:
# This is True if each row is NA.
firmyear[firmyear['size_emp_change'].isna()]

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
3,72,Google,2016,NaN


In [21]:
# We can also use ~ to negate the condition after it.
# So, here is not not NA (same as is NA).
firmyear[~firmyear['size_emp_change'].notna()]

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
3,72,Google,2016,NaN


# Merging

Like other software, `pandas` is great and merging data, and it as some conveniences not found in most other software.

Let's work through a simple example to see it in action.

In [22]:
# Remember our firm year data.
firmyear.head()

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0


In [23]:
stock = pd.read_csv('../data/stock.csv')
stock.head()

,price,tic,yr
0,86.13,msft,2018
1,62.79,msft,2017
2,54.32,msft,2016


What we'd like to do is merge in those Microsoft stock prices from the beginning of those years.
It's a bit contrived for an example, but it mirrors a lot of real world work.

While we know that Microsoft's ticker is MSFT, there's no way for `pandas` to know that without help.
So, to help, we'll make a lookup table using a dictionary.

In [24]:
lookup = {
    'Microsoft': 'MSFT',
    'Google': 'GOOG'
}

In [25]:
firmyear['id_ticker'] = firmyear['name'].map(lookup)
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker
0,114,Microsoft,2016,NaN,MSFT
1,124,Microsoft,2017,10.0,MSFT
2,131,Microsoft,2018,7.0,MSFT
3,72,Google,2016,NaN,GOOG
4,74,Google,2017,2.0,GOOG


In [26]:
# Let's make that lowercase.
firmyear['id_ticker'] = firmyear['id_ticker'].str.lower()
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker
0,114,Microsoft,2016,NaN,msft
1,124,Microsoft,2017,10.0,msft
2,131,Microsoft,2018,7.0,msft
3,72,Google,2016,NaN,goog
4,74,Google,2017,2.0,goog


In Stata, we would have another problem, namely that our column names for merging do not match.
With `pandas`, that's not a problem.

Note the validate parameter. This tells pandas that we have an expectation about how these data align with each other, and it should raise an exception if our expectation isn't met.
If you merge data without this parameter, and it unexpectedly grows in length, you may be unintentionally doing a many-to-many merge (which generally returns a new row for every pair of matches within the groups specified).

In [27]:
firmyear = firmyear.merge(stock, how='left', 
                          left_on=['id_ticker', 'year'],
                          right_on=['tic', 'yr'],
                          validate='1:1')

In [28]:
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker,price,tic,yr
0,114,Microsoft,2016,NaN,msft,54.32,msft,2016.0
1,124,Microsoft,2017,10.0,msft,62.79,msft,2017.0
2,131,Microsoft,2018,7.0,msft,86.13,msft,2018.0
3,72,Google,2016,NaN,goog,NaN,NaN,NaN
4,74,Google,2017,2.0,goog,NaN,NaN,NaN


# Querying

When working with content data, we often need to do some sort of a query to aggregate data that is interesting to us.

For example, let's add the an average word count of articles from some NYT data (similar to what we'll retrieve later) to our firmyear data.
We're only going to have results for 2018, as that's all the data I included.

In [29]:
msft_nyt = pd.read_csv('../data/msft_nyt.csv', index_col=False)

In [30]:
msft_nyt['pub_date'] = pd.to_datetime(msft_nyt['pub_date'])
msft_nyt.head()

,_id,byline.organization,byline.original,byline.person,document_type,headline.content_kicker,headline.kicker,headline.main,headline.name,headline.print_headline,...,pub_date,score,section_name,snippet,source,type_of_material,uri,web_url,word_count,id_ticker
0,5bbb75cb068401528a2df0f5,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Testing New Game-Streaming Service,NaN,Microsoft Testing New Game-Streaming Service,...,2018-10-08 15:20:41,297.65900,NaN,Microsoft Corp is testing a new game streaming...,Reuters,News,nyt://article/b5c2521b-0946-5436-adb1-62d71a73...,https://www.nytimes.com/reuters/2018/10/08/tec...,141,msft
1,5bbbe59b068401528a2df1ef,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft to Invest in Southeast Asian Ride-Ha...,NaN,Microsoft to Invest in Southeast Asian Ride-Ha...,...,2018-10-08 23:17:43,276.20792,NaN,Microsoft Corp is investing in Southeast Asian...,Reuters,News,nyt://article/f5290bfb-217c-5308-b763-2207b405...,https://www.nytimes.com/reuters/2018/10/08/bus...,327,msft
2,5bbb4bb5068401528a2df0a8,REUTERS,By REUTERS,[],article,NaN,NaN,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,NaN,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,...,2018-10-08 12:21:08,271.93472,NaN,U.S. software giant Microsoft is set to win un...,Reuters,News,nyt://article/928a254e-c6eb-5eca-896c-3556a5fa...,https://www.nytimes.com/reuters/2018/10/08/bus...,174,msft
3,5bbcabf1068401528a2df379,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Expands Cloud Service in Push for $1...,NaN,Microsoft Expands Cloud Service in Push for $1...,...,2018-10-09 13:23:59,270.83615,NaN,Microsoft Corp said on Tuesday its expanded Az...,Reuters,News,nyt://article/94179203-ee24-5ce5-bb96-641ee251...,https://www.nytimes.com/reuters/2018/10/09/bus...,158,msft
4,5bb295c6068401528a2ddeeb,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Co-Founder Paul Allen Treated Again ...,NaN,Microsoft Co-Founder Paul Allen Treated Again ...,...,2018-10-01 21:46:45,257.48560,NaN,Microsoft Corp co-founder Paul Allen said on M...,Reuters,News,nyt://article/2f19d5d8-3def-5d8d-9ba8-519385d2...,https://www.nytimes.com/reuters/2018/10/01/us/...,273,msft


In [31]:
_AGG = {
    'word_count': ['mean', 'sum']
}

def query_docs(data, ticker, year):
    summary = (data[(data['id_ticker'] == ticker) & 
                    (data['pub_date'].dt.year == year)].agg(_AGG)
                     .T.reset_index(drop=True))
    summary['id_ticker'] = ticker
    summary['year'] = year
    summary = summary.rename(columns={'mean': 'wc_mean', 'sum': 'wc_sum'})
    return summary

In [32]:
results = pd.DataFrame()
for index, row in firmyear.loc[:, ['id_ticker', 'year']].iterrows():
    results = results.append(query_docs(msft_nyt, row['id_ticker'], row['year']))

In [33]:
results

,wc_mean,wc_sum,id_ticker,year
0,NaN,0.0,msft,2016
0,NaN,0.0,msft,2017
0,519.0,5190.0,msft,2018
0,NaN,0.0,goog,2016
0,NaN,0.0,goog,2017
0,NaN,0.0,goog,2018


In [34]:
firmyear = firmyear.merge(results, how='left',
                          on=['id_ticker', 'year'],
                          validate='1:1')
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker,price,tic,yr,wc_mean,wc_sum
0,114,Microsoft,2016,NaN,msft,54.32,msft,2016.0,NaN,0.0
1,124,Microsoft,2017,10.0,msft,62.79,msft,2017.0,NaN,0.0
2,131,Microsoft,2018,7.0,msft,86.13,msft,2018.0,519.0,5190.0
3,72,Google,2016,NaN,goog,NaN,NaN,NaN,NaN,0.0
4,74,Google,2017,2.0,goog,NaN,NaN,NaN,NaN,0.0


# Saving and exporting

pandas is able to write data in a number of formats that you may need.
You can see a [reference](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) in the user guide.

Two in particular merit an additional mention:

1. **Parquet.** Apache Parquet is a high-performance compressed file format that I like to use for data that I want to use again in Python. It retains the type information, and it continues to work well up to file sizes of a few GBs.
1. **SQL.** If you are working with a database directly (including the ones we will see later), the SQL support in pandas is really convenient. That said, if you are using a service with its own package (e.g., WRDS), you probably want the more specific package.

# Breakout Exercises

Let's do two exercises to reinforce the concepts we learned above.


1. functions
1. loops

## EX1: try your data

Let's use pandas on a dataset you already have.

1. Read your dataset into a pandas dataframe with the name `my_data`. To find the proper function, you may want to look at the [pandas IO reference](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).
1. Display the first 10 rows.
1. Display the datatypes of the columns. Notice any problems.
1. Try some of the skills we learned above. For example, you might rename a column or select the data where a certain column takes a value (or satisfies some condition).

In [35]:
# 1-1 code


In [36]:
# 1-2 code


In [37]:
# 1-3 code


In [38]:
# 1-4 code


# Bonus content

One thing to notice in our code above is that we have several datasets all in memory at once.
In some stats packages, this is not nearly so easy.

For example, in Stata, they recently added the concept of multiple datasets, but the interface is much more difficult to use.
In contrast, with pandas, we simply use the name of the dataframe and then whatever operation that we are doing.

In [39]:
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker,price,tic,yr,wc_mean,wc_sum
0,114,Microsoft,2016,NaN,msft,54.32,msft,2016.0,NaN,0.0
1,124,Microsoft,2017,10.0,msft,62.79,msft,2017.0,NaN,0.0
2,131,Microsoft,2018,7.0,msft,86.13,msft,2018.0,519.0,5190.0
3,72,Google,2016,NaN,goog,NaN,NaN,NaN,NaN,0.0
4,74,Google,2017,2.0,goog,NaN,NaN,NaN,NaN,0.0


In [40]:
stock.head()

,price,tic,yr
0,86.13,msft,2018
1,62.79,msft,2017
2,54.32,msft,2016


In [41]:
msft_nyt.head()

,_id,byline.organization,byline.original,byline.person,document_type,headline.content_kicker,headline.kicker,headline.main,headline.name,headline.print_headline,...,pub_date,score,section_name,snippet,source,type_of_material,uri,web_url,word_count,id_ticker
0,5bbb75cb068401528a2df0f5,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Testing New Game-Streaming Service,NaN,Microsoft Testing New Game-Streaming Service,...,2018-10-08 15:20:41,297.65900,NaN,Microsoft Corp is testing a new game streaming...,Reuters,News,nyt://article/b5c2521b-0946-5436-adb1-62d71a73...,https://www.nytimes.com/reuters/2018/10/08/tec...,141,msft
1,5bbbe59b068401528a2df1ef,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft to Invest in Southeast Asian Ride-Ha...,NaN,Microsoft to Invest in Southeast Asian Ride-Ha...,...,2018-10-08 23:17:43,276.20792,NaN,Microsoft Corp is investing in Southeast Asian...,Reuters,News,nyt://article/f5290bfb-217c-5308-b763-2207b405...,https://www.nytimes.com/reuters/2018/10/08/bus...,327,msft
2,5bbb4bb5068401528a2df0a8,REUTERS,By REUTERS,[],article,NaN,NaN,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,NaN,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,...,2018-10-08 12:21:08,271.93472,NaN,U.S. software giant Microsoft is set to win un...,Reuters,News,nyt://article/928a254e-c6eb-5eca-896c-3556a5fa...,https://www.nytimes.com/reuters/2018/10/08/bus...,174,msft
3,5bbcabf1068401528a2df379,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Expands Cloud Service in Push for $1...,NaN,Microsoft Expands Cloud Service in Push for $1...,...,2018-10-09 13:23:59,270.83615,NaN,Microsoft Corp said on Tuesday its expanded Az...,Reuters,News,nyt://article/94179203-ee24-5ce5-bb96-641ee251...,https://www.nytimes.com/reuters/2018/10/09/bus...,158,msft
4,5bb295c6068401528a2ddeeb,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Co-Founder Paul Allen Treated Again ...,NaN,Microsoft Co-Founder Paul Allen Treated Again ...,...,2018-10-01 21:46:45,257.48560,NaN,Microsoft Corp co-founder Paul Allen said on M...,Reuters,News,nyt://article/2f19d5d8-3def-5d8d-9ba8-519385d2...,https://www.nytimes.com/reuters/2018/10/01/us/...,273,msft
